In [10]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action="ignore")
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic' # 예시: 맑은 고딕으로 설정 (Windows 환경 시)
plt.rcParams['axes.unicode_minus'] = False

**2022~2023년 합본 데이터셋 가공해서 새로 저장하기**

In [11]:
data = pd.read_csv("C:/건강검진/health_checkup/JDM/2022년~2023년/건강데이터_2022_2023_합본.csv")
# data=pd.read_csv("C:/semi_project/health_checkup/JDM/Real/processed_dataset_real2.csv")

In [14]:
import os
print(os.getcwd())  # 현재 디렉토리 확인

c:\건강검진\health_checkup\JDM\Real


In [12]:
import pandas as pd

# 남기고 싶은 컬럼 리스트
columns_to_keep = [
    'ID','year','age','sex','HE_ht', 'HE_wt', 'HE_wc', 'HE_BMI',
    'HE_sbp1', 'HE_dbp1',
    'HE_Upro', 'HE_glu', 'HE_chol', 'HE_HDL_st2',
    'HE_TG', 'HE_LDL_drct', 'HE_crea', 'HE_ast', 'HE_alt',
    # 'L_OUT_FQ', 'BP1','BD1_11', 'BS3_2', 'BE5_1', 
    'HE_obe', 'HE_HP', 'HE_DM_HbA1c',
    'DE1_dg', 'DI1_dg'
]

# 원래 데이터프레임이 df일 때
data = data[columns_to_keep]

# HE_DM_HbA1c
if 'HE_DM_HbA1c' in data.columns:
    data['HE_DM_HbA1c'] = data['HE_DM_HbA1c'].replace([9, 99, 999, 888, 9.0, 99.0, 999.0, 888.0], np.nan)
    if 'DE1_dg' in data.columns:
        print(f"DE1_dg missing ratio: {data['DE1_dg'].isnull().mean()}")
        data['HE_DM_HbA1c'] = data['HE_DM_HbA1c'].fillna(data['DE1_dg'].map({0: 1, 1: 3}))
    # 남은 결측치는 클래스 비율 기반 랜덤 샘플링
    if data['HE_DM_HbA1c'].isnull().any():
        class_probs = data['HE_DM_HbA1c'].value_counts(normalize=True)
        data['HE_DM_HbA1c'] = data['HE_DM_HbA1c'].apply(
            lambda x: np.random.choice(class_probs.index, p=class_probs.values) if pd.isna(x) else x
        )
    print(f"HE_DM_HbA1c NaN count after: {data['HE_DM_HbA1c'].isnull().sum()}")

# HE_obe
if 'HE_obe' in data.columns:
    data['HE_obe'] = data['HE_obe'].replace([9, 99, 999, 888, 9.0, 99.0, 999.0, 888.0], np.nan)
    if 'HE_BMI' in data.columns:
        # BMI가 KNN 대체된 경우 표시
        data['HE_BMI_imputed'] = data['HE_BMI'].isnull().astype(int)
        data['HE_obe'] = data.apply(
            lambda row: row['HE_obe'] if pd.notna(row['HE_obe']) else (
                1 if row['HE_BMI'] < 18.5 else
                2 if row['HE_BMI'] < 25 else
                3 if row['HE_BMI'] < 30 else
                4 if row['HE_BMI'] < 35 else
                5 if row['HE_BMI'] < 40 else
                6
            ),
            axis=1
        )
    # 남은 결측치는 클래스 비율 기반 랜덤 샘플링
    if data['HE_obe'].isnull().any():
        class_probs = data['HE_obe'].value_counts(normalize=True)
        data['HE_obe'] = data['HE_obe'].apply(
            lambda x: np.random.choice(class_probs.index, p=class_probs.values) if pd.isna(x) else x
        )
    print(f"HE_obe NaN count after: {data['HE_obe'].isnull().sum()}")

# HE_HP
if 'HE_HP' in data.columns:
    data['HE_HP'] = data['HE_HP'].replace([9, 99, 999, 888, 9.0, 99.0, 999.0, 888.0], np.nan)
    if 'DI1_dg' in data.columns:
        print(f"DI1_dg missing ratio: {data['DI1_dg'].isnull().mean()}")
        data['HE_HP'] = data['HE_HP'].fillna(data['DI1_dg'].map({0: 1, 1: 4}))
    # 남은 결측치는 클래스 비율 기반 랜덤 샘플링
    if data['HE_HP'].isnull().any():
        class_probs = data['HE_HP'].value_counts(normalize=True)
        data['HE_HP'] = data['HE_HP'].apply(
            lambda x: np.random.choice(class_probs.index, p=class_probs.values) if pd.isna(x) else x
        )
    print(f"HE_HP NaN count after: {data['HE_HP'].isnull().sum()}")


#결측치 비율 낮은 변수 처리
low_missing_columns = [
    'DI1_dg', 
    'DE1_dg', 
]
continuous_columns = ['HE_wt', 'HE_ht', 'HE_BMI']  # 연속형 변수

for col in low_missing_columns + continuous_columns:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')
        data[col] = data[col].replace([9, 99, 999, 88, 888, 9.0, 99.0, 999.0, 88.0, 888.0], np.nan)
        if not data[col].isnull().all():
            if col in continuous_columns:
                median_value = data[col].median()
                data[col] = data[col].fillna(median_value)
                print(f"Median for {col}: {median_value}")
            else:
                mode_series = data[col].mode(dropna=True)
                if not mode_series.empty:
                    mode_value = mode_series[0]
                    data[col] = data[col].fillna(mode_value)
                    print(f"Mode for {col}: {mode_value}")
                else:
                    print(f"No valid mode for {col}, leaving as is.")
        else:
            print(f"Column {col} is all NaN, skipping.")


#생체 지표 결측치 처리
knn_columns = [
    'HE_sbp1', 'HE_dbp1',  'HE_wc', 'HE_glu', 'HE_chol',
    'HE_HDL_st2', 'HE_TG', 'HE_LDL_drct',  'HE_crea', 'HE_Upro',
    'HE_ast', 'HE_alt', 'HE_wt', 'HE_ht', 'HE_BMI'
]
knn_data = data[knn_columns].copy()
knn_data = knn_data.replace([888, 999, 888.0, 999.0], np.nan)

# 스케일링
scaler = StandardScaler()
knn_data_scaled = scaler.fit_transform(knn_data)
knn_data_scaled = pd.DataFrame(knn_data_scaled, columns=knn_columns, index=knn_data.index)

# KNN Imputation
imputer_knn = KNNImputer(n_neighbors=5, weights="distance")
knn_imputed = imputer_knn.fit_transform(knn_data_scaled)

# 역스케일링
knn_imputed = scaler.inverse_transform(knn_imputed)
knn_imputed_data = pd.DataFrame(knn_imputed, columns=knn_columns, index=knn_data.index)
data[knn_columns] = knn_imputed_data

data.to_csv("건강데이터_2022_2023_합본(가공된).csv", index=False, encoding='utf-8-sig')

DE1_dg missing ratio: 0.0006821282401091405
HE_DM_HbA1c NaN count after: 0
HE_obe NaN count after: 0
DI1_dg missing ratio: 0.0006821282401091405
HE_HP NaN count after: 0
Mode for DI1_dg: 0.0
Mode for DE1_dg: 0.0
Median for HE_wt: 60.9
Median for HE_ht: 161.8
Median for HE_BMI: 23.1958666462144


In [16]:
data = pd.read_csv("C:/건강검진/health_checkup/JDM/Real/건강데이터_2022_2023_합본(가공된).csv")

1. **모델학습 및 사용할 피처 PKI 파일 다운 받기**(불균형 잡으려고 클래스 가중치 계산 시도 중)

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight

# 피처 정의 (HE_HP와 HE_DM_HbA1c 결합)
HE_HP_input_features = ['HE_sbp1', 'HE_dbp1',  'HE_wc', 'HE_glu', 'HE_BMI', 'HE_TG', 'HE_crea', 'HE_chol', 'HE_LDL_drct', 'HE_HDL_st2', 'age', 'sex']
HE_DM_HbA1c_input_features = ['HE_glu', 'HE_wc', 'HE_sbp1', 'HE_BMI', 'HE_TG', 'HE_chol', 'HE_LDL_drct', 'HE_HDL_st2', 'age', 'sex'] 
HE_obe_input_features= [ 'HE_wc', 'HE_wt','HE_sbp1', 'HE_ht', 'HE_dbp1',  'HE_glu', 'HE_crea', 'HE_TG', 'HE_HDL_st2','age','sex']

# 데이터 로드
data=pd.read_csv("C:/semi_project/health_checkup/JDM/Real/건강데이터_2022_2023_합본(가공된).csv")

# HE_HP 데이터 처리
X_hp = data[HE_HP_input_features]
y_hp = data['HE_HP'].astype(int) - 1

# HE_DM_HbA1c 데이터 처리
X_dm = data[HE_DM_HbA1c_input_features]
y_dm = data['HE_DM_HbA1c'].astype(int) - 1

# HE_obe 데이터 처리
X_obe = data[HE_obe_input_features].copy()
y_obe = data['HE_obe'].astype(int) - 1



# 훈련/테스트 분할 (HE_HP)
X_train_hp, X_test_hp, y_train_hp, y_test_hp = train_test_split(X_hp, y_hp, stratify=y_hp, test_size=0.2, random_state=42)

# 훈련/테스트 분할 (HE_DM_HbA1c)
X_train_dm, X_test_dm, y_train_dm, y_test_dm = train_test_split(X_dm, y_dm, stratify=y_dm, test_size=0.2, random_state=42)

# 훈련/테스트 분할 (HE_obe)
X_train_obe, X_test_obe, y_train_obe, y_test_obe = train_test_split(X_obe, y_obe, stratify=y_obe, test_size=0.2, random_state=42)

# 스케일링 (HE_HP)
scaler_hp = StandardScaler()
X_train_hp_scaled = scaler_hp.fit_transform(X_train_hp)
X_test_hp_scaled = scaler_hp.transform(X_test_hp)

# 스케일링 (HE_DM_HbA1c)
scaler_dm = StandardScaler()
X_train_dm_scaled = scaler_dm.fit_transform(X_train_dm)
X_test_dm_scaled = scaler_dm.transform(X_test_dm)

# 스케일링 (HE_obe)
scaler_obe = StandardScaler()
X_train_obe_scaled = scaler_obe.fit_transform(X_train_obe)
X_test_obe_scaled = scaler_obe.transform(X_test_obe)

# SMOTE 적용 (HE_HP)
smote_hp = SMOTE(random_state=42)
X_train_hp_smote, y_train_hp_smote = smote_hp.fit_resample(X_train_hp_scaled, y_train_hp)

# SMOTE 적용 (HE_DM_HbA1c)
smote_dm = SMOTE(random_state=42)
X_train_dm_smote, y_train_dm_smote = smote_dm.fit_resample(X_train_dm_scaled, y_train_dm)

# SMOTE 적용 (HE_obe)
smote_obe = SMOTE(random_state=42)
X_train_obe_smote, y_train_obe_smote = smote_obe.fit_resample(X_train_obe_scaled, y_train_obe)

# 클래스 가중치 계산 (HE_HP)
classes = np.unique(y_train_hp) # 원본 훈련 데이터의 클래스 레이블
weights = compute_class_weight('balanced', classes=classes, y=y_train_hp)
# 계산된 가중치를 딕셔너리 형태로 변환 {클래스_라벨: 가중치}
class_weights_dict_hp = dict(zip(classes, weights))

# 클래스 가중치 계산 (HE_DM_HbA1c)
classes = np.unique(y_train_dm) # 원본 훈련 데이터의 클래스 레이블
weights = compute_class_weight('balanced', classes=classes, y=y_train_dm)
# 계산된 가중치를 딕셔너리 형태로 변환 {클래스_라벨: 가중치}
class_weights_dict_dm = dict(zip(classes, weights))

# 클래스 가중치 계산 (HE_obe)
classes = np.unique(y_train_obe) # 원본 훈련 데이터의 클래스 레이블
weights = compute_class_weight('balanced', classes=classes, y=y_train_obe)
# 계산된 가중치를 딕셔너리 형태로 변환 {클래스_라벨: 가중치}
class_weights_dict_obe = dict(zip(classes, weights))


# 모델 정의 (HistGradientBoostingClassifier)
base_model_hp = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=100,
    max_depth=5,
    min_samples_leaf=20,
    l2_regularization=1.0,
    random_state=0,
    class_weight=class_weights_dict_hp
)

base_model_dm = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=100,
    max_depth=5,
    min_samples_leaf=20,
    l2_regularization=1.0,
    random_state=0,
    class_weight=class_weights_dict_dm
)

base_model_obe = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_iter=50,
    # max_depth=5,
    # min_samples_leaf=20,
    # l2_regularization=1.0,
    random_state=0,
    class_weight=class_weights_dict_obe
)

#모델 훈련
#(HE_HP)
model_hp = CalibratedClassifierCV(base_model_hp, method='sigmoid', cv=5)
model_hp.fit(X_train_hp_smote, y_train_hp_smote)

#(HE_DM_HbA1c)
model_dm = CalibratedClassifierCV(base_model_dm, method='sigmoid', cv=5)
model_dm.fit(X_train_dm_smote, y_train_dm_smote)

#(HE-obe)
model_obe = CalibratedClassifierCV(base_model_obe, method='sigmoid', cv=5)
model_obe.fit(X_train_obe_smote, y_train_obe_smote)
#base_model_obe.fit(X_train_obe_smote, y_train_obe_smote)

#테스트 데이터 평가
#(HE_HP)
y_pred_hp = model_hp.predict(X_test_hp_scaled)
print("HE_HP Classification Report:")
print(classification_report(y_test_hp, y_pred_hp, target_names=['Normal', 'Pre-hp', 'Pre-high-hp', 'High-hp']))
#(HE_DM_HbA1c)
y_pred_dm = model_dm.predict(X_test_dm_scaled)
print("HE_DM_HbA1c Classification Report:")
print(classification_report(y_test_dm, y_pred_dm, target_names=['Class 0 (Normal)', 'Class 1 (Intermediate)', 'Class 2 (Diabetes)']))
#(HE_obe)
y_pred_obe = model_obe.predict(X_test_obe_scaled)
print("HE_obe Classification Report:")
print(classification_report(y_test_obe, y_pred_obe, target_names=['Low', 'Normal', 'Pre-obese', 'Obese1', 'Obese2', 'Obese3']))

# 모델 저장 
#(HE_HP)
calibrated_filename_hp = "For_HE_HP_calibrated_real.pkl"
joblib.dump(model_hp, calibrated_filename_hp)
print(f"HE_HP 모델이 '{calibrated_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
calibrated_filename_dm = "For_HE_DM_HbA1c_calibrated_real.pkl"
joblib.dump(model_dm, calibrated_filename_dm)
print(f"HE_DM_HbA1c 모델이 '{calibrated_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
calibrated_filename_obe = "For_HE_obe_calibrated_real.pkl"
joblib.dump(model_obe, calibrated_filename_obe)
print(f"HE_DM_HbA1c 모델이 '{calibrated_filename_obe}' 파일로 저장되었습니다.")


# 전처리 객체 저장 
#(HE_HP)
scaler_filename_hp = "HE_HP_scaler_real.pkl"
joblib.dump(scaler_hp, scaler_filename_hp)
print(f"HE_HP 스케일러가 '{scaler_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
scaler_filename_dm = "HE_DM_HbA1c_scaler_real.pkl"
joblib.dump(scaler_dm, scaler_filename_dm)
print(f"HE_DM_HbA1c 스케일러가 '{scaler_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
scaler_filename_obe = "HE_obe_scaler_real.pkl"
joblib.dump(scaler_obe, scaler_filename_obe)
print(f"HE_DM_HbA1c 스케일러가 '{scaler_filename_obe}' 파일로 저장되었습니다.")


# 피처 목록 저장 
#(HE_HP)
features_filename_hp = "HE_HP_input_features_list_real.pkl"
joblib.dump(HE_HP_input_features, features_filename_hp)
print(f"HE_HP 피처 목록이 '{features_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
features_filename_dm = "HE_DM_HbA1c_input_features_list_real.pkl"
joblib.dump(HE_DM_HbA1c_input_features, features_filename_dm)
print(f"HE_DM_HbA1c 피처 목록이 '{features_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
features_filename_obe = "HE_obe_input_features_list_real.pkl"
joblib.dump(HE_obe_input_features, features_filename_obe)
print(f"HE_DM_HbA1c 피처 목록이 '{features_filename_obe}' 파일로 저장되었습니다.")


# 훈련 데이터 평균값 저장
# (HE_HP)
train_filename_hp = "train_means_hp_real.pkl"
joblib.dump(X_train_hp.mean(), train_filename_hp)
print(f"HE_HP 훈련 데이터 평균값이 '{train_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
train_filename_dm = "train_means_dm_real.pkl"
joblib.dump(X_train_dm.mean(), train_filename_dm)
print(f"HE_DM_HbA1c 훈련 데이터 평균값이 '{train_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
train_filename_obe = "train_means_obe_real.pkl"
joblib.dump(X_train_dm.mean(), train_filename_obe)
print(f"HE_obe 훈련 데이터 평균값이 '{train_filename_obe}' 파일로 저장되었습니다.")


HE_HP Classification Report:
              precision    recall  f1-score   support

      Normal       0.83      0.76      0.79      1171
      Pre-hp       0.27      0.52      0.36       207
 Pre-high-hp       0.43      0.58      0.50       359
     High-hp       0.72      0.56      0.63       902

    accuracy                           0.65      2639
   macro avg       0.56      0.60      0.57      2639
weighted avg       0.70      0.65      0.66      2639

HE_DM_HbA1c Classification Report:
                        precision    recall  f1-score   support

      Class 0 (Normal)       0.85      0.84      0.84      1438
Class 1 (Intermediate)       0.73      0.64      0.68       794
    Class 2 (Diabetes)       0.55      0.71      0.62       407

              accuracy                           0.76      2639
             macro avg       0.71      0.73      0.71      2639
          weighted avg       0.77      0.76      0.76      2639

HE_obe Classification Report:
              precis

2. **저장된 PKI 불러와 사용자 입력 값을 받을 시 예측 리포트 제공**

In [9]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV

# 사용자 입력 처리 함수
def process_user_input(user_data, features, scaler, train_means):
    user_df = pd.DataFrame([user_data]).reindex(columns=features)
    user_df = user_df.fillna(train_means)  # 누락된 피처를 평균값으로 채움
    user_scaled = scaler.transform(user_df)
    return user_df, user_scaled

# 예측 함수
def predict_disease_probabilities(user_scaled, model, stages, class_explanations):
    probs = model.predict_proba(user_scaled)[0]
    stage_index = np.argmax(probs)
    stage = stages.get(stage_index, "Unknown")
    explanation = class_explanations.get(stage_index, "No explanation available.")
    class_probs_formatted = [f"{probs[i]:.2f}" for i in range(len(probs))]
    return {
        "probabilities": class_probs_formatted,
        "predicted_class": stage_index + 1,
        "class_name": stage,
        "explanation": explanation
    }

# 각 클래스의 설명
diabetes_class_explanations = {
    0: "혈당 수준이 정상입니다. 당뇨병 위험이 낮습니다.",
    1: "혈당이 정상보다 높은 수준이지만 당뇨병에는 이르지 않았습니다. 생활습관 개선이 필요합니다.",
    2: "당뇨병으로 진단되었습니다. 전문가와 상담이 필요합니다."
}

hypertension_class_explanations = {
    0: "혈압이 정상 범위에 있습니다.",
    1: "혈압이 정상보다 높지만 고혈압 전단계로 간주됩니다. 건강 관리가 필요합니다.",
    2: "고혈압 전단계로, 생활습관 개선과 치료가 필요할 수 있습니다.",
    3: "고혈압, 전문가의 진단과 치료가 필요합니다."
}

obesity_class_explanations = {
    0: "저체중입니다. 체중 증가가 필요할 수 있습니다.",
    1: "정상 체중입니다. 유지 관리가 필요합니다.",
    2: "과체중입니다. 체중 감량을 고려할 수 있습니다.",
    3: "비만입니다. 체중 감량과 건강 관리가 필요합니다.",
    4: "고도 비만입니다. 전문가의 진단과 관리가 필요합니다.",
    5: "초고도 비만입니다. 전문가의 도움을 받아야 합니다."
}

user_data = {
    'HE_glu': 90.0,       # 공복 혈당 100~125 → 경계형 당뇨
    'HE_HbA1c': 6.0,       # HbA1c 약간 높은 수준 (참고)
    'HE_ht': 170, 
    'HE_wt': 63,           # 키 170에 체중 63이면 BMI 약 21.8 → 정상
    'HE_wc': 78, 
    'HE_BMI': 21.8,
    'HE_sbp1': 150,        # 수축기 혈압 140 이상 → 고혈압 유지
    'HE_dbp1': 90,
    'HE_sbp2': 140, 
    'HE_dbp2': 85,
    'HE_Upro': 3,
    'HE_chol': 200, 
    'HE_HDL_st2': 55,
    'HE_TG': 120, 
    'HE_LDL_drct': 110, 
    'HE_crea': 0.85, 
    'HE_ast': 22, 
    'HE_alt': 18,
    'LS_FRUOT': 5
}


# 모델, 스케일러, 피처, 평균값 로드
model_dm = joblib.load('For_HE_DM_HbA1c_calibrated_real.pkl')
scaler_dm = joblib.load('HE_DM_HbA1c_scaler_real.pkl')
features_dm = joblib.load('HE_DM_HbA1c_input_features_list_real.pkl')
train_means_dm = joblib.load('train_means_dm_real.pkl')

model_hp = joblib.load('For_HE_HP_calibrated_real.pkl')
scaler_hp = joblib.load('HE_HP_scaler_real.pkl')
features_hp = joblib.load('HE_HP_input_features_list_real.pkl')
train_means_hp = joblib.load('train_means_hp_real.pkl')

model_obesity = joblib.load('For_HE_obe_calibrated_real.pkl')
scaler_obesity = joblib.load('HE_obe_scaler_real.pkl')
features_obesity = joblib.load('HE_obe_input_features_list_real.pkl')
train_means_obesity = joblib.load('train_means_obe_real.pkl')



# 사용자 입력 처리 및 예측
user_df_dm, user_scaled_dm = process_user_input(user_data, features_dm, scaler_dm, train_means_dm)
user_df_hp, user_scaled_hp = process_user_input(user_data, features_hp, scaler_hp, train_means_hp)
user_df_obesity, user_scaled_obesity = process_user_input(user_data, features_obesity, scaler_obesity, train_means_obesity)

# 당뇨병 예측
diabetes_stages = {0: "정상", 1: "경계형 당뇨", 2: "당뇨병"}
result_dm = predict_disease_probabilities(user_scaled_dm, model_dm, diabetes_stages, diabetes_class_explanations)

# 고혈압 예측
hypertension_stages = {0: "정상 혈압", 1: "주의 혈압 단계", 2: "고혈압 전단계", 3: "고혈압"}
result_hp = predict_disease_probabilities(user_scaled_hp, model_hp, hypertension_stages, hypertension_class_explanations)

# 비만도 예측
obesity_stages = {0: "저체중", 1: "정상 체중", 2: "과체중", 3: "비만", 4: "고도 비만", 5: "초고도 비만"}
result_obesity = predict_disease_probabilities(user_scaled_obesity, model_obesity, obesity_stages, obesity_class_explanations)


def generate_disease_report(user_name, disease_name, result, top_feature_name, user_value):
    # ✅ 질병별 정상 클래스 인덱스 설정
    normal_class_index = {
        "당뇨병": 0,
        "고혈압": 0,
        "비만": 1
    }

    # ✅ 건강 점수 계산 (정상 클래스 확률 기준)
    normal_index = normal_class_index.get(disease_name, 0)
    prob_percent = float(result['probabilities'][normal_index]) * 100
    health_score = round(prob_percent, 1)

    # ✅ 예측된 클래스 확률 (예측 결과 옆에 표시)
    predicted_index = result['predicted_class'] - 1
    predicted_prob = float(result['probabilities'][predicted_index]) * 100

    # 변수 이름 매핑 (설명 추가)
    feature_label_map = {
        "HE_sbp1": "HE_sbp1 (수축기 혈압)",
        "HE_glu": "HE_glu (공복혈당)",
        "HE_BMI": "HE_BMI(BMI)"
    }
    feature_label = feature_label_map.get(top_feature_name, top_feature_name)

    # 기준 범위 설정
    if top_feature_name == "HE_sbp1":
        if user_value <= 120:
            status = "정상"
        elif 121 <= user_value <= 139:
            status = "주의"
        else:
            status = "고위험"
        normal = "90~120"
        warning = "121~139"
    elif top_feature_name == "HE_glu":
        if user_value <= 99:
            status = "정상"
        elif 100 <= user_value <= 125:
            status = "주의"
        else:
            status = "고위험"
        normal = "70~99"
        warning = "100~125"
    elif top_feature_name == "HE_BMI":
        if user_value < 18.5:
            status = "저체중"
        elif 18.5 <= user_value <= 22.9:
            status = "정상"
        elif 23.0 <= user_value <= 24.9:
            status = "과체중"
        else:
            status = "비만"
        normal = "18.5~22.9"
        warning = "23.0~24.9 이상"
    else:
        status = "알 수 없음"
        normal = "-"
        warning = "-"

    # ✅ 출력
    print(f"=== {disease_name} 예측 결과 ===")
    # 예측 결과가 정상일 때만 문구를 다르게 표시
    if result['class_name'] == "정상" or result['class_name'] == "정상 혈압" or result['class_name'] == "정상 체중":
        print(f"1. {user_name}님, 현재 {disease_name} 관련 상태는 정상으로 예측됩니다.\n")
    else:
        print(f"1. {user_name}님, 건강은 {disease_name} 위험이 있습니다.\n")
    print(f"2. 건강 점수: {health_score:.1f}점")
    print(f"예측 결과: {result['class_name']}, 예측 확률: {predicted_prob:.1f}%)")
    print(f"설명: {result['explanation']}\n")

    print(f"3. {feature_label} 수치: {user_value} → 상태: {status}")
    print(f"정상 기준: {normal}, 주의 기준: {warning}")
    print(f"이 수치가 높아지면 {disease_name} 위험이 증가합니다.\n")

# 대표 feature 맵
top_features = {
    "당뇨병": "HE_glu",
    "고혈압": "HE_sbp1",
    "비만": "HE_BMI"
}


# 질병별 결과를 묶기
disease_results = {
    "당뇨병": result_dm,
    "고혈압": result_hp,
    "비만": result_obesity
}

    # 자동 리포트 출력
for disease_name, result in disease_results.items():
    top_feature = top_features[disease_name]
    user_value = user_data.get(top_feature, None)

    if user_value is not None:
        generate_disease_report(
            user_name="홍길동",
            disease_name=disease_name,
            result=result,
            top_feature_name=top_feature,
            user_value=user_value
        )
    else:
        print(f"{disease_name}에 대한 {top_feature} 값이 user_data에 없습니다.")


=== 당뇨병 예측 결과 ===
1. 홍길동님, 현재 당뇨병 관련 상태는 정상으로 예측됩니다.

2. 건강 점수: 80.0점
예측 결과: 정상, 예측 확률: 80.0%)
설명: 혈당 수준이 정상입니다. 당뇨병 위험이 낮습니다.

3. HE_glu (공복혈당) 수치: 90.0 → 상태: 정상
정상 기준: 70~99, 주의 기준: 100~125
이 수치가 높아지면 당뇨병 위험이 증가합니다.

=== 고혈압 예측 결과 ===
1. 홍길동님, 건강은 고혈압 위험이 있습니다.

2. 건강 점수: 0.0점
예측 결과: 고혈압, 예측 확률: 72.0%)
설명: 고혈압, 전문가의 진단과 치료가 필요합니다.

3. HE_sbp1 (수축기 혈압) 수치: 150 → 상태: 고위험
정상 기준: 90~120, 주의 기준: 121~139
이 수치가 높아지면 고혈압 위험이 증가합니다.

=== 비만 예측 결과 ===
1. 홍길동님, 현재 비만 관련 상태는 정상으로 예측됩니다.

2. 건강 점수: 96.0점
예측 결과: 정상 체중, 예측 확률: 96.0%)
설명: 정상 체중입니다. 유지 관리가 필요합니다.

3. HE_BMI(BMI) 수치: 21.8 → 상태: 정상
정상 기준: 18.5~22.9, 주의 기준: 23.0~24.9 이상
이 수치가 높아지면 비만 위험이 증가합니다.



In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV

# 사용자 입력 처리 함수
def process_user_input(user_data, features, scaler, train_means):
    user_df = pd.DataFrame([user_data]).reindex(columns=features)
    user_df = user_df.fillna(train_means)  # 누락된 피처를 평균값으로 채움
    user_scaled = scaler.transform(user_df)
    return user_df, user_scaled

# 예측 함수
def predict_disease_probabilities(user_scaled, model, stages, class_explanations):
    probs = model.predict_proba(user_scaled)[0]
    stage_index = np.argmax(probs)
    stage = stages.get(stage_index, "Unknown")
    explanation = class_explanations.get(stage_index, "No explanation available.")
    class_probs_formatted = [f"{probs[i]:.2f}" for i in range(len(probs))]
    return {
        "probabilities": class_probs_formatted,
        "predicted_class": stage_index + 1,
        "class_name": stage,
        "explanation": explanation
    }

# 각 클래스의 설명
diabetes_class_explanations = {
    0: "혈당 수준이 정상입니다. 당뇨병 위험이 낮습니다.",
    1: "혈당이 정상보다 높은 수준이지만 당뇨병에는 이르지 않았습니다. 생활습관 개선이 필요합니다.",
    2: "당뇨병으로 진단되었습니다. 전문가와 상담이 필요합니다."
}

hypertension_class_explanations = {
    0: "혈압이 정상 범위에 있습니다.",
    1: "혈압이 정상보다 높지만 고혈압 전단계로 간주됩니다. 건강 관리가 필요합니다.",
    2: "고혈압 전단계로, 생활습관 개선과 치료가 필요할 수 있습니다.",
    3: "고혈압, 전문가의 진단과 치료가 필요합니다."
}

obesity_class_explanations = {
    0: "저체중입니다. 체중 증가가 필요할 수 있습니다.",
    1: "정상 체중입니다. 유지 관리가 필요합니다.",
    2: "과체중입니다. 체중 감량을 고려할 수 있습니다.",
    3: "비만입니다. 체중 감량과 건강 관리가 필요합니다.",
    4: "고도 비만입니다. 전문가의 진단과 관리가 필요합니다.",
    5: "초고도 비만입니다. 전문가의 도움을 받아야 합니다."
}

user_data = {
    'HE_glu': 90.0,       # 공복 혈당 100~125 → 경계형 당뇨
    'HE_HbA1c': 6.0,       # HbA1c 약간 높은 수준 (참고)
    'HE_ht': 170, 
    'HE_wt': 63,           # 키 170에 체중 63이면 BMI 약 21.8 → 정상
    'HE_wc': 78, 
    'HE_BMI': 21.8,
    'HE_sbp1': 150,        # 수축기 혈압 140 이상 → 고혈압 유지
    'HE_dbp1': 90,
    'HE_sbp2': 140, 
    'HE_dbp2': 85,
    'HE_Upro': 3,
    'HE_chol': 200, 
    'HE_HDL_st2': 55,
    'HE_TG': 120, 
    'HE_LDL_drct': 110, 
    'HE_crea': 0.85, 
    'HE_ast': 22, 
    'HE_alt': 18,
    'LS_FRUOT': 5
}


# 모델, 스케일러, 피처, 평균값 로드
model_dm = joblib.load('For_HE_DM_HbA1c_calibrated_real.pkl')
scaler_dm = joblib.load('HE_DM_HbA1c_scaler_real.pkl')
features_dm = joblib.load('HE_DM_HbA1c_input_features_list_real.pkl')
train_means_dm = joblib.load('train_means_dm_real.pkl')

model_hp = joblib.load('For_HE_HP_calibrated_real.pkl')
scaler_hp = joblib.load('HE_HP_scaler_real.pkl')
features_hp = joblib.load('HE_HP_input_features_list_real.pkl')
train_means_hp = joblib.load('train_means_hp_real.pkl')

model_obesity = joblib.load('For_HE_obe_calibrated_real.pkl')
scaler_obesity = joblib.load('HE_obe_scaler_real.pkl')
features_obesity = joblib.load('HE_obe_input_features_list_real.pkl')
train_means_obesity = joblib.load('train_means_obe_real.pkl')



# 사용자 입력 처리 및 예측
user_df_dm, user_scaled_dm = process_user_input(user_data, features_dm, scaler_dm, train_means_dm)
user_df_hp, user_scaled_hp = process_user_input(user_data, features_hp, scaler_hp, train_means_hp)
user_df_obesity, user_scaled_obesity = process_user_input(user_data, features_obesity, scaler_obesity, train_means_obesity)

# 당뇨병 예측
diabetes_stages = {0: "정상", 1: "경계형 당뇨", 2: "당뇨병"}
result_dm = predict_disease_probabilities(user_scaled_dm, model_dm, diabetes_stages, diabetes_class_explanations)

# 고혈압 예측
hypertension_stages = {0: "정상 혈압", 1: "주의 혈압 단계", 2: "고혈압 전단계", 3: "고혈압"}
result_hp = predict_disease_probabilities(user_scaled_hp, model_hp, hypertension_stages, hypertension_class_explanations)

# 비만도 예측
obesity_stages = {0: "저체중", 1: "정상 체중", 2: "과체중", 3: "비만", 4: "고도 비만", 5: "초고도 비만"}
result_obesity = predict_disease_probabilities(user_scaled_obesity, model_obesity, obesity_stages, obesity_class_explanations)


def generate_disease_report(user_name, disease_name, result, top_feature_name, user_value):
    # ✅ 질병별 정상 클래스 인덱스 설정
    normal_class_index = {
        "당뇨병": 0,
        "고혈압": 0,
        "비만": 1
    }

    # ✅ 건강 점수 계산 (정상 클래스 확률 기준)
    normal_index = normal_class_index.get(disease_name, 0)
    prob_percent = float(result['probabilities'][normal_index]) * 100
    health_score = round(prob_percent, 1)

    # ✅ 예측된 클래스 확률 (예측 결과 옆에 표시)
    predicted_index = result['predicted_class'] - 1
    predicted_prob = float(result['probabilities'][predicted_index]) * 100

    # 변수 이름 매핑 (설명 추가)
    feature_label_map = {
        "HE_sbp1": "HE_sbp1 (수축기 혈압)",
        "HE_glu": "HE_glu (공복혈당)",
        "HE_BMI": "HE_BMI(BMI)"
    }
    feature_label = feature_label_map.get(top_feature_name, top_feature_name)

    # 기준 범위 설정
    if top_feature_name == "HE_sbp1":
        if user_value <= 120:
            status = "정상"
        elif 121 <= user_value <= 139:
            status = "주의"
        else:
            status = "고위험"
        normal = "90~120"
        warning = "121~139"
    elif top_feature_name == "HE_glu":
        if user_value <= 99:
            status = "정상"
        elif 100 <= user_value <= 125:
            status = "주의"
        else:
            status = "고위험"
        normal = "70~99"
        warning = "100~125"
    elif top_feature_name == "HE_BMI":
        if user_value < 18.5:
            status = "저체중"
        elif 18.5 <= user_value <= 22.9:
            status = "정상"
        elif 23.0 <= user_value <= 24.9:
            status = "과체중"
        else:
            status = "비만"
        normal = "18.5~22.9"
        warning = "23.0~24.9 이상"
    else:
        status = "알 수 없음"
        normal = "-"
        warning = "-"
        
            # ✅ 클래스별 이름 매핑
    class_labels_map = {
        "당뇨병": ["정상", "경계형 당뇨", "당뇨병"],
        "고혈압": ["정상 혈압", "주의 혈압 단계", "고혈압 전단계", "고혈압"],
        "비만": ["저체중", "정상 체중", "과체중", "비만", "고도 비만", "초고도 비만"]
    }
    class_labels = class_labels_map.get(disease_name, [f"클래스 {i}" for i in range(len(result["probabilities"]))])

    # ✅ 각 클래스별 확률 텍스트 생성
    class_prob_text = ", ".join([
        f"{label} 확률: {float(prob) * 100:.0f}%"
        for label, prob in zip(class_labels, result["probabilities"])
    ])

    # ✅ 출력
    print(f"=== {disease_name} 예측 결과 ===")

    if result['class_name'] in ["정상", "정상 혈압", "정상 체중"]:
        print(f"1. {user_name}님, 현재 {disease_name} 관련 상태는 정상으로 예측됩니다.")
    else:
        print(f"1. {user_name}님, 건강은 {disease_name} 위험이 있습니다.")
    
    print(f"   ({class_prob_text})\n")

    print(f"2. 건강 점수: {health_score:.1f}점 (이 점수는 정상 확률 기준입니다.)\n")

    print(f"3. {feature_label} 수치: {user_value} → 상태: {status}")
    print(f"정상 기준: {normal}, 주의 기준: {warning}")
    print(f"{disease_name}과 가장 높게 연관된 {feature_label.split()[0]}은(는) 높아지면 {disease_name} 위험이 증가합니다.\n")

    return health_score  # ✅ 점수 반환


# 대표 feature 맵
top_features = {
    "당뇨병": "HE_glu",
    "고혈압": "HE_sbp1",
    "비만": "HE_BMI"
}


# 질병별 결과를 묶기
disease_results = {
    "당뇨병": result_dm,
    "고혈압": result_hp,
    "비만": result_obesity
}

# ✅ 전체 건강 점수 저장용 리스트
health_scores = []

# ✅ 자동 리포트 출력
for disease_name, result in disease_results.items():
    top_feature = top_features[disease_name]
    user_value = user_data.get(top_feature, None)

    if user_value is not None:
        score = generate_disease_report(
            user_name="홍길동",
            disease_name=disease_name,
            result=result,
            top_feature_name=top_feature,
            user_value=user_value
        )
        health_scores.append(score)
    else:
        print(f"{disease_name}에 대한 {top_feature} 값이 user_data에 없습니다.")

# ✅ 총점 출력은 딱 한 번, 루프 끝나고!
if health_scores:
    total_score = sum(health_scores) / len(health_scores)
    print(f"총 개인 건강 점수: {total_score:.1f}점")
    print("(당뇨병, 고혈압, 비만에 대한 정상상태 점수를 평균한 값입니다.)")
    #당뇨병의로 진단되었습니다.
    #전문가와 상담이 필요합니다.
    #예측결과 : Diatabetes (클래스3)


=== 당뇨병 예측 결과 ===
1. 홍길동님, 현재 당뇨병 관련 상태는 정상으로 예측됩니다.
   (정상 확률: 81%, 경계형 당뇨 확률: 18%, 당뇨병 확률: 0%)

2. 건강 점수: 81.0점 (이 점수는 정상 확률 기준입니다.)

3. HE_glu (공복혈당) 수치: 90.0 → 상태: 정상
정상 기준: 70~99, 주의 기준: 100~125
당뇨병과 가장 높게 연관된 HE_glu은(는) 높아지면 당뇨병 위험이 증가합니다.

=== 고혈압 예측 결과 ===
1. 홍길동님, 건강은 고혈압 위험이 있습니다.
   (정상 혈압 확률: 0%, 주의 혈압 단계 확률: 0%, 고혈압 전단계 확률: 28%, 고혈압 확률: 72%)

2. 건강 점수: 0.0점 (이 점수는 정상 확률 기준입니다.)

3. HE_sbp1 (수축기 혈압) 수치: 150 → 상태: 고위험
정상 기준: 90~120, 주의 기준: 121~139
고혈압과 가장 높게 연관된 HE_sbp1은(는) 높아지면 고혈압 위험이 증가합니다.

=== 비만 예측 결과 ===
1. 홍길동님, 현재 비만 관련 상태는 정상으로 예측됩니다.
   (저체중 확률: 0%, 정상 체중 확률: 98%, 과체중 확률: 2%, 비만 확률: 0%, 고도 비만 확률: 0%, 초고도 비만 확률: 0%)

2. 건강 점수: 98.0점 (이 점수는 정상 확률 기준입니다.)

3. HE_BMI(BMI) 수치: 21.8 → 상태: 정상
정상 기준: 18.5~22.9, 주의 기준: 23.0~24.9 이상
비만과 가장 높게 연관된 HE_BMI(BMI)은(는) 높아지면 비만 위험이 증가합니다.

총 개인 건강 점수: 59.7점
(당뇨병, 고혈압, 비만에 대한 정상상태 점수를 평균한 값입니다.)


In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
import shap # shap 라이브러리 추가
import matplotlib.pyplot as plt

# 사용자 입력 처리 함수
def process_user_input(user_data, features, scaler, train_means):
    user_df = pd.DataFrame([user_data]).reindex(columns=features)
    user_df = user_df.fillna(train_means)  # 누락된 피처를 평균값으로 채움
    user_scaled = scaler.transform(user_df)
    return user_df, user_scaled

# 예측 함수
def predict_disease_probabilities(user_scaled, model, stages, class_explanations):
    probs = model.predict_proba(user_scaled)[0]
    stage_index = np.argmax(probs)
    stage = stages.get(stage_index, "Unknown")
    explanation = class_explanations.get(stage_index, "No explanation available.")
    class_probs_formatted = [f"{probs[i]:.2f}" for i in range(len(probs))]
    return {
        "probabilities": class_probs_formatted,
        "predicted_class": stage_index + 1,
        "class_name": stage,
        "explanation": explanation
    }

# 각 클래스의 설명
diabetes_class_explanations = {
    0: "혈당 수준이 정상입니다. 당뇨병 위험이 낮습니다.",
    1: "혈당이 정상보다 높은 수준이지만 당뇨병에는 이르지 않았습니다. 생활습관 개선이 필요합니다.",
    2: "당뇨병으로 진단되었습니다. 전문가와 상담이 필요합니다."
}

hypertension_class_explanations = {
    0: "혈압이 정상 범위에 있습니다.",
    1: "혈압이 정상보다 높지만 고혈압 전단계로 간주됩니다. 건강 관리가 필요합니다.",
    2: "고혈압 전단계로, 생활습관 개선과 치료가 필요할 수 있습니다.",
    3: "고혈압, 전문가의 진단과 치료가 필요합니다."
}

obesity_class_explanations = {
    0: "저체중입니다. 체중 증가가 필요할 수 있습니다.",
    1: "정상 체중입니다. 유지 관리가 필요합니다.",
    2: "과체중입니다. 체중 감량을 고려할 수 있습니다.",
    3: "비만입니다. 체중 감량과 건강 관리가 필요합니다.",
    4: "고도 비만입니다. 전문가의 진단과 관리가 필요합니다.",
    5: "초고도 비만입니다. 전문가의 도움을 받아야 합니다."
}

# 의학적 의미 딕셔너리 정의 (SHAP 설명에 사용)
medical_meanings = {
    'HE_glu': "당뇨병 진단의 핵심 기준이며 혈당 조절 상태를 나타내는 중요한 지표",
    'age': "2형 당뇨병 발병의 강력한 위험 인자",
    'HE_chol': "총 콜레스테롤은 이상지질혈증의 지표로, 당뇨와 관련된 대사증후군의 구성 요소",
    'HE_LDL_drct': "LDL 콜레스테롤은 이상지질혈증의 지표로, 당뇨와 관련된 대사증후군의 구성 요소",
    'HE_wc': "허리둘레는 복부 비만 및 대사증후군의 지표",
    'HE_BMI': "체질량지수는 비만/과체중의 지표로 당뇨 위험 증가와 관련",
    'HE_sbp1': "수축기 혈압은 고혈압 진단 기준 중 하나이며 심혈관 질환 위험과 관련",
    'HE_dbp1': "이완기 혈압은 고혈압 진단 기준 중 하나이며 심혈관 질환 위험과 관련",
    'HE_crea': "크레아티닌은 신장 기능 및 근육량과 관련되며, 비만/대사증후군과 연관될 수 있는 지표", # 상세 설명 추가
    'HE_ast': "AST(간 기능 수치)는 간세포 손상 또는 비알코올성 지방간 질환(NAFLD)과 연관될 수 있으며, NAFLD는 대사증후군/고혈압/당뇨 위험을 높입니다.", # 상세 설명 추가
    'HE_alt': "ALT(간 기능 수치)는 간세포 손상 또는 비알코올성 지방간 질환(NAFLD)과 연관될 수 있으며, NAFLD는 대사증후군/고혈압/당뇨 위험을 높입니다.", # 상세 설명 추가
    'HE_TG': "중성지방은 이상지질혈증의 지표로, 대사증후군/심혈관 질환/당뇨 위험과 관련",
    'HE_HDL_st2': "HDL 콜레스테롤은 이상지질혈증의 지표로, 낮은 수치는 대사증후군/심혈관 질환/당뇨 위험을 높입니다."
    # 다른 변수들의 의미도 필요에 따라 추가
}



user_data = {
    'HE_glu': 90.0,       # 공복 혈당 100~125 → 경계형 당뇨
    'HE_HbA1c': 6.0,       # HbA1c 약간 높은 수준 (참고)
    'HE_ht': 170, 
    'HE_wt': 63,           # 키 170에 체중 63이면 BMI 약 21.8 → 정상
    'HE_wc': 78, 
    'HE_BMI': 21.8,
    'HE_sbp1': 150,        # 수축기 혈압 140 이상 → 고혈압 유지
    'HE_dbp1': 90,
    'HE_sbp2': 140, 
    'HE_dbp2': 85,
    'HE_Upro': 3,
    'HE_chol': 200, 
    'HE_HDL_st2': 55,
    'HE_TG': 120, 
    'HE_LDL_drct': 110, 
    'HE_crea': 0.85, 
    'HE_ast': 22, 
    'HE_alt': 18,
    'LS_FRUOT': 5
}


# 모델, 스케일러, 피처, 평균값 로드
model_dm = joblib.load('For_HE_DM_HbA1c_calibrated_real.pkl')
scaler_dm = joblib.load('HE_DM_HbA1c_scaler_real.pkl')
features_dm = joblib.load('HE_DM_HbA1c_input_features_list_real.pkl')
train_means_dm = joblib.load('train_means_dm_real.pkl')

model_hp = joblib.load('For_HE_HP_calibrated_real.pkl')
scaler_hp = joblib.load('HE_HP_scaler_real.pkl')
features_hp = joblib.load('HE_HP_input_features_list_real.pkl')
train_means_hp = joblib.load('train_means_hp_real.pkl')

model_obesity = joblib.load('For_HE_obe_calibrated_real.pkl')
scaler_obesity = joblib.load('HE_obe_scaler_real.pkl')
features_obesity = joblib.load('HE_obe_input_features_list_real.pkl')
train_means_obesity = joblib.load('train_means_obe_real.pkl')



# 사용자 입력 처리 및 예측
user_df_dm, user_scaled_dm = process_user_input(user_data, features_dm, scaler_dm, train_means_dm)
user_df_hp, user_scaled_hp = process_user_input(user_data, features_hp, scaler_hp, train_means_hp)
user_df_obesity, user_scaled_obesity = process_user_input(user_data, features_obesity, scaler_obesity, train_means_obesity)

# 당뇨병 예측
diabetes_stages = {0: "정상", 1: "경계형 당뇨", 2: "당뇨병"}
result_dm = predict_disease_probabilities(user_scaled_dm, model_dm, diabetes_stages, diabetes_class_explanations)

# 고혈압 예측
hypertension_stages = {0: "정상 혈압", 1: "주의 혈압 단계", 2: "고혈압 전단계", 3: "고혈압"}
result_hp = predict_disease_probabilities(user_scaled_hp, model_hp, hypertension_stages, hypertension_class_explanations)

# 비만도 예측
obesity_stages = {0: "저체중", 1: "정상 체중", 2: "과체중", 3: "비만", 4: "고도 비만", 5: "초고도 비만"}
result_obesity = predict_disease_probabilities(user_scaled_obesity, model_obesity, obesity_stages, obesity_class_explanations)


def generate_disease_report(user_name, disease_name, result, top_feature_name, user_value):
    # ✅ 질병별 정상 클래스 인덱스 설정
    normal_class_index = {
        "당뇨병": 0,
        "고혈압": 0,
        "비만": 1
    }

    # ✅ 건강 점수 계산 (정상 클래스 확률 기준)
    normal_index = normal_class_index.get(disease_name, 0)
    prob_percent = float(result['probabilities'][normal_index]) * 100
    health_score = round(prob_percent, 1)

    # ✅ 예측된 클래스 확률 (예측 결과 옆에 표시)
    predicted_index = result['predicted_class'] - 1
    predicted_prob = float(result['probabilities'][predicted_index]) * 100

    # 변수 이름 매핑 (설명 추가)
    feature_label_map = {
        "HE_sbp1": "HE_sbp1 (수축기 혈압)",
        "HE_glu": "HE_glu (공복혈당)",
        "HE_BMI": "HE_BMI(BMI)"
    }
    feature_label = feature_label_map.get(top_feature_name, top_feature_name)

    # 기준 범위 설정
    if top_feature_name == "HE_sbp1":
        if user_value <= 120:
            status = "정상"
        elif 121 <= user_value <= 139:
            status = "주의"
        else:
            status = "고위험"
        normal = "90~120"
        warning = "121~139"
    elif top_feature_name == "HE_glu":
        if user_value <= 99:
            status = "정상"
        elif 100 <= user_value <= 125:
            status = "주의"
        else:
            status = "고위험"
        normal = "70~99"
        warning = "100~125"
    elif top_feature_name == "HE_BMI":
        if user_value < 18.5:
            status = "저체중"
        elif 18.5 <= user_value <= 22.9:
            status = "정상"
        elif 23.0 <= user_value <= 24.9:
            status = "과체중"
        else:
            status = "비만"
        normal = "18.5~22.9"
        warning = "23.0~24.9 이상"
    else:
        status = "알 수 없음"
        normal = "-"
        warning = "-"

    # ✅ 출력
    print(f"=== {disease_name} 예측 결과 ===")
    # 예측 결과가 정상일 때만 문구를 다르게 표시
    if result['class_name'] == "정상" or result['class_name'] == "정상 혈압" or result['class_name'] == "정상 체중":
        print(f"1. {user_name}님, 현재 {disease_name} 관련 상태는 정상으로 예측됩니다.\n")
    else:
        print(f"1. {user_name}님, 건강은 {disease_name} 위험이 있습니다.\n")
    print(f"2. 건강 점수: {health_score:.1f}점")
    print(f"예측 결과: {result['class_name']}, 예측 확률: {predicted_prob:.1f}%)")
    print(f"설명: {result['explanation']}\n")

    print(f"3. {feature_label} 수치: {user_value} → 상태: {status}")
    print(f"정상 기준: {normal}, 주의 기준: {warning}")
    print(f"이 수치가 높아지면 {disease_name} 위험이 증가합니다.\n")

# 대표 feature 맵
top_features = {
    "당뇨병": "HE_glu",
    "고혈압": "HE_sbp1",
    "비만": "HE_BMI"
}


# 질병별 결과를 묶기
disease_results = {
    "당뇨병": result_dm,
    "고혈압": result_hp,
    "비만": result_obesity
}

    # 자동 리포트 출력
for disease_name, result in disease_results.items():
    top_feature = top_features[disease_name]
    user_value = user_data.get(top_feature, None)

    if user_value is not None:
        generate_disease_report(
            user_name="홍길동",
            disease_name=disease_name,
            result=result,
            top_feature_name=top_feature,
            user_value=user_value
        )
    else:
        print(f"{disease_name}에 대한 {top_feature} 값이 user_data에 없습니다.")


In [ ]:
# import pandas as pd
# import numpy as np
# import joblib
# from sklearn.preprocessing import StandardScaler
# from imblearn.over_sampling import SMOTE
# from sklearn.ensemble import HistGradientBoostingClassifier
# from sklearn.calibration import CalibratedClassifierCV

# # 사용자 입력 처리 함수
# def process_user_input(user_data, features, scaler, train_means):
#     user_df = pd.DataFrame([user_data]).reindex(columns=features)
#     user_df = user_df.fillna(train_means)  # 누락된 피처를 평균값으로 채움
#     user_scaled = scaler.transform(user_df)
#     return user_df, user_scaled

# # 예측 함수
# def predict_disease_probabilities(user_scaled, model, stages, class_explanations):
#     probs = model.predict_proba(user_scaled)[0]
#     stage_index = np.argmax(probs)
#     stage = stages.get(stage_index, "Unknown")
#     explanation = class_explanations.get(stage_index, "No explanation available.")
#     class_probs_formatted = [f"{probs[i]:.2f}" for i in range(len(probs))]
#     return {
#         "probabilities": class_probs_formatted,
#         "predicted_class": stage_index + 1,
#         "class_name": stage,
#         "explanation": explanation
#     }

# # 각 클래스의 설명
# diabetes_class_explanations = {
#     0: "Normal: 혈당 수준이 정상입니다. 당뇨병 위험이 낮습니다.",
#     1: "Pre-diabetes: 혈당이 정상보다 높은 수준이지만 당뇨병에는 이르지 않았습니다. 생활습관 개선이 필요합니다.",
#     2: "Diabetes: 당뇨병으로 진단되었습니다. 전문가와 상담이 필요합니다."
# }

# hypertension_class_explanations = {
#     0: "Normal: 혈압이 정상 범위에 있습니다.",
#     1: "Pre-hypertension: 혈압이 정상보다 높지만 고혈압 전단계로 간주됩니다. 건강 관리가 필요합니다.",
#     2: "Hypertension Stage 1: 고혈압 전단계로, 생활습관 개선과 치료가 필요할 수 있습니다.",
#     3: "Hypertension Stage 2: 고혈압, 전문가의 진단과 치료가 필요합니다."
# }

# obesity_class_explanations = {
#     0: "Underweight: 저체중입니다. 체중 증가가 필요할 수 있습니다.",
#     1: "Normal: 정상 체중입니다. 유지 관리가 필요합니다.",
#     2: "Overweight: 과체중입니다. 체중 감량을 고려할 수 있습니다.",
#     3: "Obese: 비만입니다. 체중 감량과 건강 관리가 필요합니다.",
#     4: "Severely Obese: 심각한 비만입니다. 전문가의 진단과 관리가 필요합니다.",
#     5: "Morbidly Obese: 병적 비만입니다. 전문가의 도움을 받아야 합니다."
# }

# # 사용자 입력 예시
# user_data = {
#     'HE_glu': 130.0,       # 공복 혈당 126 이상 -> 당뇨병 확진
#     'HE_HbA1c': 7.0,       # HbA1c 6.5% 이상 -> 당뇨병 확진
#     'HE_ht': 170, 
#     'HE_wt': 80, 
#     'HE_wc': 100, 
#     'HE_BMI': 28,
#     'HE_sbp1': 140, 
#     'HE_dbp1': 85,
#     'HE_sbp2': 130, 
#     'HE_dbp2': 80,
#     'HE_Upro': 7,
#     'HE_chol': 220, 
#     'HE_HDL_st2': 50,
#     'HE_TG': 150, 
#     'HE_LDL_drct': 130, 
#     'HE_crea': 0.8, 
#     'HE_ast': 20, 
#     'HE_alt': 15,
#     'LS_FRUOT': 4
# }

# # 모델, 스케일러, 피처, 평균값 로드
# model_dm = joblib.load('For_HE_DM_HbA1c_calibrated_real.pkl')
# scaler_dm = joblib.load('HE_DM_HbA1c_scaler_real.pkl')
# features_dm = joblib.load('HE_DM_HbA1c_input_features_list_real.pkl')
# train_means_dm = joblib.load('train_means_dm_real.pkl')

# model_hp = joblib.load('For_HE_HP_calibrated_real.pkl')
# scaler_hp = joblib.load('HE_HP_scaler_real.pkl')
# features_hp = joblib.load('HE_HP_input_features_list_real.pkl')
# train_means_hp = joblib.load('train_means_hp_real.pkl')

# model_obesity = joblib.load('For_HE_obe_calibrated_real.pkl')
# scaler_obesity = joblib.load('HE_obe_scaler_real.pkl')
# features_obesity = joblib.load('HE_obe_input_features_list_real.pkl')
# train_means_obesity = joblib.load('train_means_obe_real.pkl')



# # 사용자 입력 처리 및 예측
# user_df_dm, user_scaled_dm = process_user_input(user_data, features_dm, scaler_dm, train_means_dm)
# user_df_hp, user_scaled_hp = process_user_input(user_data, features_hp, scaler_hp, train_means_hp)
# user_df_obesity, user_scaled_obesity = process_user_input(user_data, features_obesity, scaler_obesity, train_means_obesity)

# # 당뇨병 예측
# diabetes_stages = {0: "Normal", 1: "Pre-diabetes", 2: "Diabetes"}
# result_dm = predict_disease_probabilities(user_scaled_dm, model_dm, diabetes_stages, diabetes_class_explanations)

# # 고혈압 예측
# hypertension_stages = {0: "Normal", 1: "Pre-hypertension", 2: "Hypertension Stage 1", 3: "Hypertension Stage 2"}
# result_hp = predict_disease_probabilities(user_scaled_hp, model_hp, hypertension_stages, hypertension_class_explanations)

# # 비만도 예측
# obesity_stages = {0: "Underweight", 1: "Normal", 2: "Overweight", 3: "Obese", 4: "Severely Obese", 5: "Morbidly Obese"}
# result_obesity = predict_disease_probabilities(user_scaled_obesity, model_obesity, obesity_stages, obesity_class_explanations)


# def generate_disease_report(user_name, disease_name, result, top_feature_name, user_value):
#     # 건강 점수 계산
#     prob_percent = float(result['probabilities'][result['predicted_class'] - 1]) * 100
#     health_score = round(prob_percent, 1)

#     # 변수 이름 매핑 (설명 추가)
#     feature_label_map = {
#         "HE_sbp1": "HE_sbp1 (수축기 혈압)",
#         "HE_glu": "HE_glu (공복혈당)",
#         "HE_BMI": "HE_BMI(BMI)"
#     }
#     feature_label = feature_label_map.get(top_feature_name, top_feature_name)

#     # 기준 범위 설정
#     if top_feature_name == "HE_sbp1":
#         if user_value <= 120:
#             status = "정상"
#         elif 121 <= user_value <= 139:
#             status = "주의"
#         else:
#             status = "고위험"
#         normal = "90~120"
#         warning = "121~139"
#     elif top_feature_name == "HE_glu":
#         if user_value <= 99:
#             status = "정상"
#         elif 100 <= user_value <= 125:
#             status = "주의"
#         else:
#             status = "고위험"
#         normal = "70~99"
#         warning = "100~125"
#     elif top_feature_name == "HE_BMI":
#         if user_value < 18.5:
#             status = "저체중"
#         elif 18.5 <= user_value <= 22.9:
#             status = "정상"
#         elif 23.0 <= user_value <= 24.9:
#             status = "과체중"
#         else:
#             status = "비만"
#         normal = "18.5~22.9"
#         warning = "23.0~24.9 이상"
#     else:
#         status = "알 수 없음"
#         normal = "-"
#         warning = "-"

#     # 출력
#     print(f"=== {disease_name} 예측 결과 ===")
#     print(f"1. {user_name}님, 건강은 {disease_name} 위험이 있습니다.\n")
#     print(f"2. 건강 점수: {health_score:.1f}점 (예측 확률: {prob_percent:.1f}%)")
#     print(f"예측 결과: {result['class_name']} (클래스 {result['predicted_class']})")
#     print(f"설명: {result['explanation']}\n")

#     print(f"3. {feature_label} 수치: {user_value} → 상태: {status}")
#     print(f"정상 기준: {normal}, 주의 기준: {warning}")
#     print(f"이 수치가 높아지면 {disease_name} 위험이 증가합니다.\n")

# # 대표 feature 맵
# top_features = {
#     "당뇨병": "HE_glu",
#     "고혈압": "HE_sbp1",
#     "비만": "HE_BMI"
# }


# # 질병별 결과를 묶기
# disease_results = {
#     "당뇨병": result_dm,
#     "고혈압": result_hp,
#     "비만": result_obesity
# }

# # 자동 리포트 출력
# for disease_name, result in disease_results.items():
#     top_feature = top_features[disease_name]
#     user_value = user_data.get(top_feature, None)

#     if user_value is not None:
#         generate_disease_report(
#             user_name="홍길동",
#             disease_name=disease_name,
#             result=result,
#             top_feature_name=top_feature,
#             user_value=user_value
#         )
#     else:
#         print(f"{disease_name}에 대한 {top_feature} 값이 user_data에 없습니다.")





=== 당뇨병 예측 결과 ===
1. 홍길동님, 건강은 당뇨병 위험이 있습니다.

2. 건강 점수: 94.0점 (예측 확률: 94.0%)
예측 결과: Diabetes (클래스 3)
설명: Diabetes: 당뇨병으로 진단되었습니다. 전문가와 상담이 필요합니다.

3. HE_glu (공복혈당) 수치: 130.0 → 상태: 고위험
정상 기준: 70~99, 주의 기준: 100~125
이 수치가 높아지면 당뇨병 위험이 증가합니다.

=== 고혈압 예측 결과 ===
1. 홍길동님, 건강은 고혈압 위험이 있습니다.

2. 건강 점수: 64.0점 (예측 확률: 64.0%)
예측 결과: Hypertension Stage 1 (클래스 3)
설명: Hypertension Stage 1: 고혈압 전단계로, 생활습관 개선과 치료가 필요할 수 있습니다.

3. HE_sbp1 (수축기 혈압) 수치: 140 → 상태: 고위험
정상 기준: 90~120, 주의 기준: 121~139
이 수치가 높아지면 고혈압 위험이 증가합니다.

=== 비만 예측 결과 ===
1. 홍길동님, 건강은 비만 위험이 있습니다.

2. 건강 점수: 96.0점 (예측 확률: 96.0%)
예측 결과: Obese (클래스 4)
설명: Obese: 비만입니다. 체중 감량과 건강 관리가 필요합니다.

3. HE_BMI(BMI) 수치: 28 → 상태: 비만
정상 기준: 18.5~22.9, 주의 기준: 23.0~24.9 이상
이 수치가 높아지면 비만 위험이 증가합니다.

